In [1]:
%cd "D:\project\helmet_object"

D:\project\helmet_object


In [2]:
from ultralytics import YOLO 

In [3]:
import optuna

d:\project\helmet_object\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import yaml
import mlflow 

with open(r"param.yml","r") as f:
    params =yaml.safe_load(f)

In [6]:
params

{'lr': {'min': 0.01, 'max': 0.1},
 'dp': {'min': 0.1, 'max': 0.6},
 'batch_option': [16, 32]}

In [7]:
min_hp_lr=params["lr"]["min"]
max_hp_lr=params["lr"]["max"]

min_hp_dp=params["dp"]["min"]
max_hp_dp=params["dp"]["max"]

batch_options=params["batch_option"]

In [8]:
mlflow.set_tracking_uri("file:///d:/project/helmet_object/runs/mlflow")

def goal_func(trail):

    lr=round(trail.suggest_float("lr",min_hp_lr,max_hp_lr),2)
    dropout=round(trail.suggest_float("dropout",min_hp_dp,max_hp_dp),1)
    batch=trail.suggest_categorical("batch",batch_options)
    model =YOLO("yolo11n.pt")
    
    mlflow.set_experiment("helmet_detection_exp")
    with mlflow.start_run(): 
        mlflow.log_params({"learning_rate": lr, "dropout": dropout,"batch_size" : batch})

        results = model.train(
                            data=r"D:\project\helmet_object\safety-helmet-1\data.yaml",
                            imgsz=128,
                            epochs=3,
                            batch=batch,
                            project="runs",
                            name=f"train_run_{trail.number}",
                            lr0=lr,
                            dropout=dropout,

                            verbose=False
                        )
        
        model.export(format="onnx")
        mlflow.log_artifact("yolo11n.onnx")
       
        val_results = model.val(name=f"val_run_{trail.number}")
        map50 = val_results.box.map50
        mlflow.log_metric("map50", map50)
        mlflow.end_run()
    
    return map50
    

study = optuna.create_study(direction="maximize")
study.optimize(goal_func,n_trials=7)

[I 2025-06-20 15:17:26,015] A new study created in memory with name: no-name-79662c60-8326-4678-9d38-6c92cec33eec
2025/06/20 15:17:26 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRE

New https://pypi.org/project/ultralytics/8.3.157 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (Intel Core(TM) i5-6300U 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\project\helmet_object\safety-helmet-1\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.3, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_run_03, nbs=64, nms=

train: Scanning D:\project\helmet_object\safety-helmet-1\train\labels.cache... 335 images, 0 backgrounds, 0 corrupt: 100%|██████████| 335/335 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.20.1 ms, read: 28.04.7 MB/s, size: 12.1 KB)



d:\project\helmet_object\env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning D:\project\helmet_object\safety-helmet-1\valid\labels.cache... 117 images, 0 backgrounds, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]
d:\project\helmet_object\env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.46s/it]


                   all        117        445     0.0191      0.592     0.0537     0.0254
Speed: 0.2ms preprocess, 13.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\train_run_6
MLflow: results logged to file:///d:/project/helmet_object/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (Intel Core(TM) i5-6300U 2.40GHz)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs\train_run_6\weights\best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 6, 336) (5.2 MB)

ONNX: starting export with onnx 1.14.1 opset 19...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success  4.1s, saved as 'runs\train_run_6\weights\best.onnx' (9.9 MB)

Export complete (5.1s)
Results saved to D:\project\helmet_object\runs\train_run_6\weights
Predict:         yolo predict task=detect model=runs\train_run_6\weights\best.onnx imgsz=128  
Va

val: Scanning D:\project\helmet_object\safety-helmet-1\valid\labels.cache... 117 images, 0 backgrounds, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


                   all        117        445     0.0196      0.599     0.0575     0.0268
Speed: 0.1ms preprocess, 18.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\val_run_6


[I 2025-06-20 15:31:01,407] Trial 6 finished with value: 0.05749493818182485 and parameters: {'lr': 0.010504029367131271, 'dropout': 0.18694000810136022, 'batch': 32}. Best is trial 2 with value: 0.2514023199610298.


In [10]:
best_trial = study.best_trial.number
best_model_path = f"runs/train/train_run_{best_trial}/weights/best.pt"